This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [1]:
structure_name = "LLZO_010_O_order5_off__Li_100_slab_heavy"

In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_O_order5_off__Li_100_slab_heavy
- Initial lattice height: 70.60 Å
- Number of atoms: 590


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 21.37 Å
Lowest Li slab atom: 34.22 Å
LLZO top z: 34.21 Å
Li penetration into LLZO: -0.01 Å


In [4]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 19.21 Å
Li slab thickness:   21.37 Å
LLZO top z:          34.21 Å
Lowest Li atom z:    34.22 Å
Li penetration into LLZO: -0.01 Å


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 420 atoms out of 590
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_174655/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 20:43:56    -2265.520191        0.603848
SciPyFminCG:    1 20:43:58    -2269.305258        5.964568
SciPyFminCG:    2 20:43:59    -2274.190340        6.733698
SciPyFminCG:    3 20:44:00    -2283.583517        3.618300
SciPyFminCG:    4 20:44:01    -2297.359314        3.173433
SciPyFminCG:    5 20:44:02    -2312.718458        2.881392
SciPyFminCG:    6 20:44:03    -2316.858287        1.264246
SciPyFminCG:    7 20:44:04    -2318.591163        1.359517
SciPyFminCG:    8 20:44:05    -2320.238795        1.230786
SciPyFminCG:    9 20:44:06    -2325.142024        4.745714
SciPyFminCG:   10 20:44:06    -2327.564590        5.606937
SciPyFminCG:   11 20:44:07    -2336.528170        2.441657
SciPyFminCG:   12 20:44:08    -2345.433655        6.222866
SciPyFminCG:   13 20:44:09    -2351.421425        3.109027
SciPyFminCG:   14 20:44:10    -2358.377829        3.779802
SciPyFminCG:   15 20:44:11    -2365.226340        1.889295

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_174655/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 20:45:10    -2403.965459        0.116776
FIRE:    1 20:45:10    -2403.967428        0.095798
FIRE:    2 20:45:10    -2403.971367        0.075168
FIRE:    3 20:45:11    -2403.975868        0.064587
FIRE:    4 20:45:11    -2403.980651        0.065552
FIRE:    5 20:45:11    -2403.985434        0.067895
FIRE:    6 20:45:12    -2403.990498        0.068916
FIRE:    7 20:45:12    -2403.996124        0.067744
FIRE:    8 20:45:13    -2404.003720        0.063599
FIRE:    9 20:45:13    -2404.013004        0.063030
FIRE:   10 20:45:13    -2404.024539        0.067615
FIRE:   11 20:45:14    -2404.038324        0.066854
FIRE:   12 20:45:14    -2404.054079        0.067493
FIRE:   13 20:45:15    -2404.072084        0.060309
FIRE:   14 20:45:15    -2404.093466        0.065290
FIRE:   15 20:45:15    -2404.117379        0.071201
FIRE:   16 20:45:16    -2404.143543        0.083811
FIRE:   17 20:45:16    -2404.172521        0.092630
FIRE:   18 20:

In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 21.62 Å
Lowest Li slab atom: 33.98 Å
LLZO top z: 33.88 Å
Li penetration into LLZO: -0.10 Å


In [10]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)

             Step     Time          Energy          fmax
SciPyFminCG:    0 20:45:50    -2405.670905        1.110016
SciPyFminCG:    1 20:45:51    -2407.251158        0.827389
SciPyFminCG:    2 20:45:52    -2409.406457        1.302975
SciPyFminCG:    3 20:45:53    -2410.266495        0.780264
SciPyFminCG:    4 20:45:54    -2411.536717        1.105934
SciPyFminCG:    5 20:45:55    -2412.748985        0.758340
SciPyFminCG:    6 20:45:56    -2413.983197        0.747891
SciPyFminCG:    7 20:45:57    -2414.783592        0.745745
SciPyFminCG:    8 20:45:58    -2415.811586        0.778282
SciPyFminCG:    9 20:45:59    -2416.696944        1.008443
SciPyFminCG:   10 20:46:00    -2418.276634        0.797321
SciPyFminCG:   11 20:46:01    -2419.610438        0.697649
SciPyFminCG:   12 20:46:02    -2420.641527        1.126491
SciPyFminCG:   13 20:46:03    -2421.010637        0.689028
SciPyFminCG:   14 20:46:04    -2421.458521        0.682459
SciPyFminCG:   15 20:46:05    -2421.766300        0.698922

In [11]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_174655/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 20:48:44    -2510.598049        0.111080
FIRE:    1 20:48:44    -2510.598893        0.111072
FIRE:    2 20:48:45    -2510.600300        0.111060
FIRE:    3 20:48:45    -2510.602551        0.111041
FIRE:    4 20:48:45    -2510.605083        0.111015
FIRE:    5 20:48:46    -2510.607896        0.110978
FIRE:    6 20:48:46    -2510.610991        0.110924
FIRE:    7 20:48:46    -2510.614648        0.110850
FIRE:    8 20:48:47    -2510.618868        0.110737
FIRE:    9 20:48:47    -2510.623932        0.110586
FIRE:   10 20:48:47    -2510.630121        0.110389
FIRE:   11 20:48:47    -2510.637155        0.110145
FIRE:   12 20:48:48    -2510.645313        0.109846
FIRE:   13 20:48:48    -2510.654879        0.109480
FIRE:   14 20:48:48    -2510.665851        0.109024
FIRE:   15 20:48:49    -2510.678511        0.108473
FIRE:   16 20:48:49    -2510.693140        0.107831
FIRE:   17 20:48:49    -2510.709739        0.106968
FIRE:   18 20:

In [12]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_010_O_order5_off__Li_100_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


In [13]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/cellrelaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")

Li slab thickness: 31.27 Å
Lowest Li slab atom: 33.51 Å
LLZO top z: 33.24 Å
Li penetration into LLZO: -0.27 Å
